# 一、数据集下载
通常来说数据量是 csv < Parquet < JSON < JSONL）。基本全是纯粹的文本数据，很少一部分是问答对数据，预训练阶段一般不需要问答对数据。这里以从huggingface拉取SkyPile数据集为例，下载并解压数据集。

## 1.多线程下载数据集
 使用Huggingface镜像站官方提供的hfd脚本、并结合aria2c和git-lfs来进行多线程并行拉取数据集。

```bash
export HF_ENDPOINT=https://hf-mirror.com
```

### 1.1下载hfd.sh  

```bash
wget https://hf-mirror.com/hfd/hfd.sh 
chmod a+x hfd.sh 
```

### 1.2安装aria2c

```
sudo apt install aria2   
aria2c --version
```

### 1.3安装git-lfs

```bash
sudo apt install git-lfs
git lfs install
```

### 1.4下载数据集

```bash
./hfd.sh Skywork/SkyPile-150B --dataset --tool aria2c -x 10 --local-dir ~你的路径/data/SkyPile
```

- `./hfd.sh` - 调用本地的 Bash 脚本 `hfd.sh`,用来从 Hugging Face 平台下载指定的资源。`hfd.sh` 是一个辅助工具,封装了 Hugging Face 的下载逻辑,并支持更多高级功能(如断点续传、多线程、多任务下载)
- `Skywork/SkyPile-150B` - 指定 Hugging Face 数据集的完整名称,由组织或用户名称(Skywork)和具体数据集名称(SkyPile-150B)组成
- `--dataset` - 指明要下载的是一个数据集(dataset),而不是模型(model)
- `--tool aria2c` - 指定使用的下载工具,这里是 `aria2c`,相比于 `wget` 或其他工具,`aria2c` 支持分块下载和多线程,能显著提升下载速度
- `-x 10` - 表示设置线程数为 10,线程数决定了单个文件的下载并发数,`aria2c` 会分块并同时下载多个部分。`-x` 的最大值通常受hfd脚本或工具限制,如果超过限制会报错(例如设置了-x为16)
- `--local-dir ~/autodl-tmp/MateConv/Data/SkyPile` - 指定下载文件存储的本地路径。在这里,文件将存储在 `~/autodl-tmp/MateConv/Data/SkyPile` 目录中。如果目录不存在,脚本会尝试自动创建

可以修改hfd.sh脚本中的-x,-j修改线程数和并行下载数



## 代码部分

运行[step1_dataclean.py](./step1_data/step1_dataclean.py)进行数据去重,是否有空白数据等，进行简单预处理，同时使用[split_jsonl.py](./step1_data/split_jsonl.py)将数据分成更小的部分，注意数据要用utf-8的格式，方便datajuicer清洗，这里还是以SkyPile为例，可以修改代码使用其他的数据集。这一步的另一个目的就是将数据集转换为标准的jsonl {"text":"..."}的格式，因为有些数据集的格式可能不是这种格式，需要转换一下。

# 二、多模态PDF数据集准备

pdf解析的难点：

* **布局解析困难**：PDF文件的布局可能会因为不同的作者、工具或用途而有所不同，通常具有多列文本，对于图像或表格来说，这些文本可能会突然中断，因此解析其布局是一个具有挑战性的任务。

* **格式错综复杂**：PDF文件中可能包含各种格式的内容，包括**文字**、**图像**、**表格**等，因此解析其内容需要考虑到这种多样性和复杂性。

* **复合表格**：纵向/横向合并的复杂表格，在PDF中进行抽象还原是最难处理的问题之一

* **文本、图片、表格顺序提取**：提取PDF文件中的文本、图片和表格，并确保它们的顺序正确性，是一个需要解决的重要问题。

* **文档结构还原**：还原PDF文件的文档结构，包括标题、目录等信息，是实现自动化文档处理和理解的关键步骤之一。

* **元数据提取**：在PDF中隐藏的元数据信息是RAG产品的关键数据，比如链接、目录、字体等等

* **扫描件**：PDF中如果是扫描件，就需要依靠OCR模型来进行有效的提取，这里面还会包含清晰度、模型的稳定性等等问题

* **Latex公式提取**：在一些特殊领域，PDF文本中包含了Latex等数学公式。通过完整的提取和转换是对RAG问答的有效补充


这里使用Unstructured库对pdf进行了简单处理，如果需要更符合业务场景或者更高精度，需要调整，这里只是提供了一种简单实现的方式：

Unstructured.io作为集成框架，通过strategy参数实现后端自适应切换：
* **"fast"策略**：调用PyMuPDF等轻量引擎处理规则文档

* **"hi_res"策略**：激活YOLOX目标检测模型进行布局分析，配合detectron2实现表格与图像的精准提取

* **"ocr_only"策略**：使用OCR模型进行图文识别

* **"vlm"策略**：针对极端复杂场景调用GPT-4o等多模态模型，通过视觉语义理解突破传统解析局限

## 1. 环境准备与安装

**此安装方式适用于处理纯文本：**
```bash
pip install unstructured
uv add unstructured
```

**包含本地推理能力**（支持PDF/图片OCR等）
```bash
pip install "unstructured[local-inference]"
```

**支持所有文档类型**（不含本地推理，需依赖外部API）
```bash
pip install "unstructured[all-docs]"
uv add "unstructured[all-docs]"
```

**仅安装PDF和DOCX**处理能力
```bash
pip install "unstructured[pdf,docx]"
```

Serverless API通过优化处理流程，将文档处理启动时间从30分钟缩短至3秒以内，并支持多区域横向扩展，有效提升了高并发场景下的吞吐量：

```bash
pip install unstructured-client
```

## 2.核心配置安装

#### （1）Tesseract OCR：图像文本识别

**windows安装：**

* 下载安装包：https://github.com/UB-Mannheim/tesseract/wiki
* 安装文档参考：https://developer.baidu.com/article/detail.html?id=3803413

**macOS安装：**
```bash
brew install tesseract
brew install tesseract-lang
```

**Linux安装：**
```bash
sudo apt-get install tesseract-ocr
sudo apt-get install tesseract-ocr-chi-sim  # 中文简体支持
```

**验证安装：**
```bash
tesseract --list-langs  # 查看已安装的语言包
```

#### （2）Poppler：PDF内容提取底层引擎

**macOS安装：**
```bash
brew install poppler
```

**Linux安装：**
```bash
sudo apt-get install poppler-utils
```

## 3.pdf文档解析

* 关键额外参数：

    - strategy：PDF解析策略的选择直接影响提取效果，可选 "auto"（默认）、"hi_res"、"fast"、"ocr_only"。控制解析方式。
        - **auto**（默认）：适用于无图像嵌入文本的标准PDF，解析速度快

        - **hi_res**：使用布局检测模型（例如 会调用布局检测模型 Detectron2/YOLOX/自研Chipper等）提取结构信息。

        - **fast**：快速解析，以可提取文本为主。

        - **ocr_only**：针对扫描件／图像版 PDF，只做 OCR 提取。

        - **vlm**：VLM模型，如OpenAI/Anthropic等提供的视觉语言模型

    - extract_images_in_pdf（布尔）：当 strategy 为 hi_res 时，可控制是否提取嵌入图像块。

    - extract_image_block_types（列表）：指定提取哪些类型（如 ["Image","Table"]）的图像块。

    - extract_image_block_to_payload（布尔）：是否把提取块转换为 payload（例如 base64）输出。

    - extract_image_block_output_dir（字符串）：如果不转换为 payload，可将提取图像块保存到指定目录。

    - max_partition：当使用 ocr_only 策略时，限制单个元素（文本块）最大字符长度。默认为 1500。

    - languages 或 ocr_languages：对 OCR 使用的语言包列表。

    - skip_infer_table_types：可以跳过表格类型推断以提高速度。

    - split_pdf_page：True，大文件分块处理，可实现大文件分块处理，提升解析效率

    - infer_table_structure：True，表格结构推断，是否尝试推断表格结构

In [ ]:
from unstructured.partition.pdf import partition_pdf
from typing import List
from unstructured.documents.elements import Element

# 使用partition_pdf函数检测pdf类型并解析
elements = partition_pdf(filename="甬兴证券-AI行业点评报告：海外科技巨头持续发力AI，龙头公司中报业绩亮眼.pdf",
                         strategy="hi_res", # 使用hi_res模式进行高精度解析
                         extract_images_in_pdf=True, # 提取pdf中的图片
                         extract_image_block_types=["Table","Image"], # 提取表格和图片
                         extract_image_block_output_dir="./images", # 保存图片到images目录
                         languages=["eng","zho"],
                         split_pdf_page=True, # 大文件分块处理，优化性能
                         infer_table_structure=True, # 是否尝试推断表格结构，会下载一个ocr模型
                         include_page_breaks=True) # 是否包含页码信息

## 代码部分

pdf解析代码：
[PdfProcess.py](./step1_data/PdfProcess.py)

# 三、datajuicer数据清洗

## 1.datajuicer安装

克隆仓库
```bash
git clone https://github.com/datajuicer/data-juicer.git
cd data-juicer
uv pip install -e .
```

同样也可以安装指定的额外领域的依赖

```bash
uv pip install -e ".[vision]"
```


运行脚本进行数据清洗，对数据进行筛选[run_step2_skypile.sh](minideepseek/step2_clean/run_step2_skypile.sh)

# 四、tokenizer

## 1.tokenizer+打包

对处理完的数据进行tokenizer处理，同时进行打包packing，生成bin文件存储[datapacked.py](minideepseek/tokenizer/datapacked.py)

## 2.dataset

定义数据读取方式[dataset.py](minideepseek/tokenizer/dataset.py)

# 五、模型训练

## 1.环境配置
acclerate库安装

```bash
pip install accelerate
```

配置acclerate

acclerate config